# One-class SVM における誤検出率（nu=0.5時）

scikit-learn の <b>One-class SVM</b> の、誤検出率（アノマリーでないデータがアノマリーと検出されてしまう率）を求める方法の検討になります。

## (0) 確認結果

結果から申し上げると、誤検出率は、パラメータの nu 値（0.5）とほぼ同じ結果（0.499）となり、これだけを見ると<B>「実用が困難」</B>という判断になってしまうかと存じます。

ですので、<a href="16.ipynb"><b>こちらのレポート</b></a>で、パラメータ nu=0.01 に下げた実験を行なっております。

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
    'test_daikin_conversation.csv',
    'test_ptna_conversation.csv',
    'test_septeni_conversation.csv'
]
copied_csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
copied_csv_file_paths

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]


['/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv']

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
    
    text_array.py における TF-IDF 処理では、
    以下の通り「する」をストップワード化指定しております
    
    class TextArray:
        :
        def __build_vectorizer(self):
            :
            vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b', stop_words=['する'])
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/23 PM 09:27:38 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 28272
2017/03/23 PM 09:27:38 TextArray#__init__ start
2017/03/23 PM 09:27:52 TextArray#to_vec start
2017/03/23 PM 09:27:53 TextArray#to_vec end


In [5]:
n_sample = X.shape[0]
n_feature = X.shape[1]
print("sample=%d, feature=%d" % (n_sample, n_feature))

sample=28272, feature=2748


### (2-2) 外れデータのTF-IDFベクター

<b><a href="12.ipynb">こちらで再検証した時と同じ質問文</a></b>をつかっております。

In [6]:
'''
    マイオペのプロダクション・コードと同じように、
    訓練データ作成時と同じベクトライザーを使用します。
'''
#from learning.core.training_set.text_array import TextArray
from prototype.modules.text_array import TextArray

test_X = [
    '要素技術は自然languageの機械learningですか？', # まったくfeatureが抽出されない質問文

    '人生相談をしたいのですが？', # featureが１件抽出されるな質問文「する」を含む
    '難解なプログラミング技術を調達？', # featureが１件抽出される質問文「する」を含まない

    '何か習い事をしますか？', # featureが２件抽出される質問文「する」を含む
    '何か習い事がいいですか？', # featureが２件抽出される質問文「する」を含まない
    
    '何か習い事をしたほうがいいですか？', # featureが３件抽出される質問文「する」を含む
    '何か習い事を行うほうがいいですか？', # featureが３件抽出される質問文「する」を含まない
    
    '会社を辞めたいのですが誰に相談するのがいいですか？', # featureが４件抽出される質問文「する」を含む
    '会社を辞めたいのですが誰に相談を行うのがいいですか？', # featureが４件抽出される質問文「する」を含まない

    '有給休暇を取って海外に行き旅行する意向があります。', # featureが５件抽出される質問文「する」を含む
    '有給休暇を取って海外に行きたいと思っています。', # featureが５件抽出される質問文「する」を含まない
]
vectorizer = training_set.body_array.vectorizer
text_array = TextArray(test_X, vectorizer=vectorizer)

'''
    外れデータのTF-IDFベクターを取得
'''
X_error = text_array.to_vec()

2017/03/23 PM 09:27:53 TextArray#__init__ start
2017/03/23 PM 09:27:53 TextArray#to_vec start
2017/03/23 PM 09:27:53 TextArray#to_vec end


In [7]:
vocabulary = text_array._vectorizer.vocabulary_
dumped_features = TestTool.get_dumped_features(X_error, vocabulary)
for d in dumped_features:
    print(d)

index=0[]
index=1[]
index=2[調達=1.000]
index=3[何=1.000]
index=4[いい=0.707 何=0.707]
index=5[いい=0.707 何=0.707]
index=6[いい=0.577 何=0.577 行う=0.577]
index=7[いい=0.577 会社=0.577 誰=0.577]
index=8[いい=0.500 会社=0.500 行う=0.500 誰=0.500]
index=9[休暇=0.500 取る=0.500 海外=0.500 行く=0.500]
index=10[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447]


## (3) One-class SVMで学習

線形カーネルを使用しています。パラメータはデフォルトになります。

In [8]:
'''
    One-class SVM with linear kernel, anomary is half
        fn(i,j) = i'*j
'''
from sklearn import svm
clf_linear = svm.OneClassSVM(
    kernel='linear', # linearカーネルを使用
    nu=0.5           # 外れ判定される件数の見積もりを全体の5割と想定
    ) 
clf_linear.fit(X)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

## (4) 誤検出率の算定

### (4-1) 訓練データを使って予測実行

アノマリーでないデータ＝訓練データを使って予測した時、アノマリーと検出されてしまうデータがあるかどうか確認します。

In [9]:
'''
    訓練データを使用して、外れ判定を実行してみます
'''
pred_by_linear_for_train = clf_linear.predict(X)
pred_by_linear_for_train

array([ 1., -1., -1., ..., -1., -1., -1.])

### (4-2) 誤検出率を算出

誤検出率＝アノマリー検出件数 ÷ 訓練データ件数 になります。

パラメータの nu 値（デフォルト=0.5）とほぼ同じになるため、モデルとしては正しいという事は分かります。

ただし、その事が判断基準というのはあり得ないかと存じます。

In [10]:
n_anomaly = len([c for c in pred_by_linear_for_train if c == -1])

print('誤検出率=%0.3f [サンプル件数=%d, アノマリー件数=%d]' % (n_anomaly / n_sample, n_sample, n_anomaly))

誤検出率=0.499 [サンプル件数=28272, アノマリー件数=14108]


### (4-3) 参考：アノマリーと検出された訓練データだけを抽出

In [11]:
'''
    (4-1) において、アノマリーと検出されたデータを抽出
'''
from scipy.sparse import lil_matrix, csr_matrix

X_misdetected_temp = lil_matrix((n_anomaly, n_feature))
idx = 0
for i, v in enumerate(pred_by_linear_for_train):
    if v == -1:
        X_misdetected_temp[idx] = X.getrow(i)
        idx += 1
        
X_misdetected = X_misdetected_temp.tocsr()
X_misdetected

<14108x2748 sparse matrix of type '<class 'numpy.float64'>'
	with 64664 stored elements in Compressed Sparse Row format>

### (4-4) 参考：アノマリーと検出された訓練データだけを使って予測したら、結果はどうなるか？

#### 全て外れとなります（当たり前ですが）

In [12]:
'''
    外れと判定されてしまった訓練データだけを使用して、
    再度、外れ判定を実行してみます
'''
pred_by_linear_for_train_2 = clf_linear.predict(X_misdetected)
pred_by_linear_for_train_2

array([-1., -1., -1., ..., -1., -1., -1.])

In [13]:
n_anomaly_2 = len([c for c in pred_by_linear_for_train_2 if c == -1])

print('誤検出率=%0.3f [サンプル件数=%d, アノマリー件数=%d]' % (n_anomaly_2 / n_anomaly, n_anomaly, n_anomaly_2))

誤検出率=1.000 [サンプル件数=14108, アノマリー件数=14108]


## (5) 参考：外れデータを使って予測

#### 結果として全て外れとなったのは、nu=0.5 の指定により、外れ判定が緩くなったためと推測します。

In [14]:
'''
    外れデータを使用して、外れ判定を実行してみます
'''
pred_by_linear_for_error = clf_linear.predict(X_error)
pred_by_linear_for_error

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])